#Named Entity Recognition

The subtask of information extraction that seeks to locate and classify named entity mentions in unstructured text into pre-defined categories such as the person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.”

It is very common that sequence modeling, such as HMM, MEMM, CRF, is applied to named entity prediction. In this lab 09, we will train and test the named entity prediction with sequence modeling, such as CRF. The first activity will show how to train the sequence model for NER and predict next words given some previous words.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
# read IOB tagged NER dataset as dataframe
df = pd.read_csv('https://drive.google.com/uc?id=1UPHg29593FCCNqec6RyYitjfoyY90UWN', encoding = "ISO-8859-1")
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In the data, you can see the different types of entities: 
* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon

## Data Preprocessing
There are too many NaN values in ‘Sentence #” column, fill NaN by preceding values.
We have 47595 sentences that contain 35172 unique words and tagged by 17 tags.

In [ ]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(47959, 35172, 17)

In [ ]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,402
1,B-eve,308
2,B-geo,37644
3,B-gpe,15870
4,B-nat,201
5,B-org,20143
6,B-per,16990
7,B-tim,20333
8,I-art,297
9,I-eve,253


We will now train a CRF model for named entity recognition using sklearn-crfsuite on our dataset. As mentioned before, MEMM or CRF is often used for labeling or parsing of sequential data for named entity recognition.

In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 4.0MB/s 


##Conditional random fields (CRF)

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [ ]:
#Retrieving sentences with their POS and tags.
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

We extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklearn-crfsuite format — each sentence should be converted to a list of dicts. The following code were taken from [sklearn-crfsuites official site](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html).

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
# data splitting for training and testing

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
# train a CRF model for named entity recognition using sklearn-crfsuite 
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Because tag “O” (outside) is the most common tag and it will make our results look much better than they actual are. So we remove tag “O” when we evaluate classification metrics.

In [ ]:
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
classes.pop()
classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim']

In [ ]:
#evaluation
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=classes))

              precision    recall  f1-score   support

       B-art       0.45      0.12      0.19       143
       B-eve       0.59      0.42      0.49       106
       B-geo       0.86      0.91      0.88     12447
       B-gpe       0.97      0.94      0.95      5284
       B-nat       0.82      0.42      0.56        78
       B-org       0.80      0.73      0.76      6615
       B-per       0.85      0.82      0.84      5652
       B-tim       0.93      0.88      0.90      6856
       I-art       0.11      0.03      0.05       105
       I-eve       0.38      0.22      0.28        93
       I-geo       0.82      0.81      0.81      2520
       I-gpe       0.91      0.62      0.74        69
       I-nat       1.00      0.43      0.61        23
       I-org       0.82      0.80      0.81      5597
       I-per       0.85      0.90      0.87      5674
       I-tim       0.84      0.74      0.79      2207

   micro avg       0.86      0.85      0.85     53469
   macro avg       0.75   

The following shows what our classifier learned. It is very likely that the beginning of a geographical entity (B-geo) will be followed by a token inside geographical entity (I-geo), but transitions to inside of an organization name (I-org) from tokens with other labels are penalized hugely.

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-nat  -> I-nat   6.934503
I-art  -> I-art   6.260215
B-art  -> I-art   5.881224
I-eve  -> I-eve   5.847777
B-eve  -> I-eve   5.586673
I-tim  -> I-tim   5.204188
I-org  -> I-org   4.782243
I-gpe  -> I-gpe   4.699609
B-tim  -> I-tim   4.636703
B-org  -> I-org   4.282602
O      -> O       3.813956
B-per  -> I-per   3.698815
I-geo  -> I-geo   3.685166
B-gpe  -> I-gpe   3.597376
B-geo  -> I-geo   3.516476
I-per  -> I-per   3.245863
I-nat  -> I-nat   2.954009
I-geo  -> B-art   1.973397
O      -> B-tim   1.748999
O      -> B-per   1.620428

Top unlikely transitions:
I-org  -> I-geo   -4.259782
I-org  -> I-per   -4.327937
B-geo  -> B-geo   -4.426926
B-per  -> I-org   -4.427218
B-geo  -> I-gpe   -4.435073
B-per  -> I-geo   -4.466408
B-tim  -> B-tim   -4.518613
B-org  -> I-geo   -4.575173
B-geo  -> I-per   -4.793920
B-org  -> I-per   -5.036090
B-geo  -> I-org   -5.070524
B-gpe  -> I-geo   -5.210003
B-gpe  -> I-org   -5.287803
B-gpe  -> B-gpe   -5.607401
O      -> I-per  

#NER and Coreference Resolution with Spacy 

Now, we will learn how to predict NER and Coreference Resolution with [Spacy](https://spacy.io/)

In [ ]:
!pip install -U spacy==2.1.3
!python -m spacy download en

     |████████████████████████████████| 27.7MB 122kB/s 
     |████████████████████████████████| 2.1MB 41.8MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 3.2MB 32.6MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.3 which is incompatible.
  Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 11.1MB 4.1MB/s 
  Created wheel for en-core-web-s

In [ ]:
from pprint import pprint

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm


##Named Entity Recognition with Spacy

The following lines show how to build named entity recognizer with SpaCy, to identify the names of things, such as persons, organizations, or locations. SpaCy’s named entity recognition has been trained on the [OntoNotes 5 corpus](https://catalog.ldc.upenn.edu/LDC2013T19) and it supports the following entity types: https://spacy.io/api/annotation#section-named-entities

In [ ]:
# loading pre-trained model of NER
nlp = en_core_web_sm.load()

In [ ]:
!pip install wikipedia
import wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=c09da9a268d2656747c2533570e8c02e4efd9cca99cf7e61f9d7a4dac7b8517a
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


We will extract a wikipedia page (with Barack Obama) to test NER with Spacy. There are 2087 entities in the page.

In [ ]:
# getting wikipedia page of Barack obama
wikip = wikipedia.page("Barack_Obama")
article=nlp(wikip.content)
len(article.ents)

2171

In [ ]:
# count the number of entitie types found from wikipedia page
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'CARDINAL': 101,
         'DATE': 504,
         'EVENT': 19,
         'FAC': 21,
         'GPE': 390,
         'LANGUAGE': 3,
         'LAW': 28,
         'LOC': 23,
         'MONEY': 30,
         'NORP': 135,
         'ORDINAL': 61,
         'ORG': 415,
         'PERCENT': 37,
         'PERSON': 369,
         'PRODUCT': 7,
         'QUANTITY': 4,
         'TIME': 4,
         'WORK_OF_ART': 20})

In [ ]:
# getting the top 10 words recognised as named entity
items = [x.text for x in article.ents]
Counter(items).most_common(10)

[('Obama', 221),
 ('U.S.', 51),
 ('first', 43),
 ('the United States', 28),
 ('American', 17),
 ('Chicago', 16),
 ('Senate', 16),
 ('2009', 13),
 ('Republican', 13),
 ('Iraq', 13)]

In [ ]:
sentences = [x for x in article.sents]
print(sentences[0])

Barack Hussein Obama II ( (listen) bə-RAHK hoo-SAYN oh-BAH-mə;


In [ ]:
# display each tag of the sentence
print([(x, x.ent_iob_, x.ent_type_) for x in sentences[0]])

[(Barack, 'B', 'PERSON'), (Hussein, 'I', 'PERSON'), (Obama, 'I', 'PERSON'), (II, 'I', 'PERSON'), ((, 'O', ''), ((, 'O', ''), (listen, 'O', ''), (), 'O', ''), (bə, 'O', ''), (-, 'O', ''), (RAHK, 'O', ''), (hoo, 'O', ''), (-, 'O', ''), (SAYN, 'O', ''), (oh, 'O', ''), (-, 'O', ''), (BAH, 'O', ''), (-, 'O', ''), (mə, 'O', ''), (;, 'O', '')]


In [ ]:
# display whole sentences using render()
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

## Coreference Resolution with mention-ranking model and Spacy

This coreference resolution module is based on the spaCy parser and uses the neural net **mention-ranking scoring model** described in [Deep Reinforcement Learning for Mention-Ranking Coreference Models](https://cs.stanford.edu/people/kevclark/resources/clark-manning-emnlp2016-deep.pdf) by Kevin Clark and Christopher D. Manning, EMNLP 2016. 

In [ ]:
!pip install neuralcoref

     |████████████████████████████████| 286kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 133kB 42.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 7.5MB 38.2MB/s 
ERROR: botocore 1.20.69 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import neuralcoref

100%|██████████| 40155833/40155833 [00:04<00:00, 8791734.43B/s] 


In [ ]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')

# Load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

# You can now use NeuralCoref with the wikipedia content that we extracted in the previous section and it's annotations.
doc = nlp(wikip.content)

doc._.has_coref
doc._.coref_clusters

[the United States: [the United States, the United States],
 Obama: [Obama, He, Obama, he, he, he, he, he, he, Obama, his, his, his, he, his, Obama, his, he, Obama, his, he, he, He, He, he, His, He, he, Obama, Obama, Obama, his, his, Obama, his, his, his, His, Obama, his, Obama, his, Obama, Obama, Obama, his, Obama, his, he, he, He, Obama, he, Obama, his, he, Obama, he, his, Obama, his, Obama, he, his, Obama, his, He, Obama, Obama, Obama, Obama, Obama, he, he, he, He, his, he, Obama, He, he, his, he, Obama, Obama, Obama, Obama, Obama, Obama, Obama, his, He, he, He, his, He, his, his, his, Obama, his, he, Obama, Obama, his, Obama, Obama, Obama, Obama, he, He, He, Obama, he, he, his, Obama, He, his, his, his, he, he, he, Obama, Obama, Obama, Obama, Obama, Obama, Obama, He, Obama, He, he, Obama, He, his, Obama, his, Obama, his, Obama, his, He, Obama, his, Obama, He, his, Obama, him, his, Obama, His, his, Obama, Obama, Obama, him, Obama, he, his, his, Senator Obama, Obama, Obama, its, Obam

#Bi-LSTM CRF 
Here, we introduce a pytorch version of NER on Bi-LSTM CRF from Pytorch official document: https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html. 

Because each time we only feed one sentence into the model, the sequence length for the input of LSTM can be different. Pytorch will handle this automatically.

## Example

In [ ]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

### Helper functions to make the code more readable.

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

### Create model

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

### Run training

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

(tensor(2.6907), [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
(tensor(20.4906), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])


# Exercise




## E1. Explain the difference between coreference and anaphora with the example. (Your answer will not be marked if you do not add the example)

Your answer: Coreferece refers to the case when 2 or more phrases refer to the same entity. E.g. **Kevin Durant** is a 2 times Finals MVP. There is no doubt that **KD** is definitely one of the best NBA player.

Anaphora happens when a word refer to (part of) an entity mentioned by an earlier word/phrase in the text. E.g. Although **question E1** explicitly says "our answer will not be marked without *the* example", **it** is still ambiguous because it is unclear what **the phrase in the question "the example"** refers to.  

## E2. Try Bi-LSTM with CRF!

Now we will apply the Bi-LSTM CRF model we just learned to CoNLL 2003 NER dataset using the pretrained glove embeddings. Please go through and complete the [Function for accuracy] section. 

### Download Dataset

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
drive = None
def authenticate():
    global drive
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
    authenticate()
    for fileId in fileIds:    
        downloaded = drive.CreateFile({"id": fileId[1]})
        downloaded.GetContentFile(fileId[0])

In [2]:
#Download file if not existing
try:
  _ = open("train.txt", "r")
except:
  downloadFiles([["train.txt", "1UmNHdUZxjfcuIzCcAKuBvfBXdSWFv47i"]])

try:
  _ = open("validation.txt", "r")
except:
  downloadFiles([["validation.txt", "11bZIh5V9m2nZJ5s5xQ_gxHEHkAEhV8eQ"]])

try:
  _ = open("test.txt", "r")
except:
  downloadFiles([["test.txt", "1V-LQuJWT62aCytYuhZuaxvICsqiF1rdK"]])

In [3]:
def read_data(file_name, n_sample):
    f = open(file_name)
    documents = f.readlines()

    input_data = []
    target_data = []

    temp1 = []
    temp2 = []
    for i in documents:
        if i == '\n':
            input_data.append(temp1)
            target_data.append(temp2)
            temp1 = []
            temp2 = []
        else:
            temp1.append(i.replace('\n','').split(' ')[0].lower())
            temp2.append(i.replace('\n','').split(' ')[3])
    return input_data[:n_sample], target_data[:n_sample]

train_data, target_y_train = read_data("train.txt",400)
validation_data, target_y_validation = read_data("validation.txt",50)
test_data, target_y_test = read_data("test.txt",50)

In [4]:
print(len(train_data))
print(train_data[1])
print(target_y_train[1])

400
['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']


### Preprocess

#### Generate word_to_ix and tag_to_ix

In [5]:
word_to_ix = {}
for sentence in train_data+validation_data+test_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train+target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


#### Generate Embedding Matrix

In [ ]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25") 

EMBEDDING_DIM = 25

In [7]:
import numpy as np
embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


(2351, 25)

#### convert dataset into idxs

In [8]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(train_data,word_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)
test_output_index = to_index(target_y_test,tag_to_ix)

### Model

In [9]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

#### Function for accuracy [Please Complete this part]

Please complete the *cal_acc* function that generates the model predictions using the input data and calculates the accuracy by comparing the model predictions with the ground truth labels. You can refer to the [Train the model] section regarding what the inputs and outputs are and how they will be used.

**Hint**: You are going to test the "model" using "input_index"(whose shape is (num_samples, seq_length)), and you are going to return three variables:
- **predicted**: This should be a list, and each item in the list should be the predicted NER label for a word. Suppose you have two sentences, sentence 1 has 8 words and sentence 2 has 6 words. This list should have 14(=8+6) items.
- **ground_truth**: This should also be a list, and each item in the list should be the actual NER label for a word, which can be easily gotten from "output_index". Suppose you have two sentences, sentence 1 has 8 words and sentence 2 has 6 words. This list should have 14(=8+6) items.
- **accuracy**: You are going to use "predicted" and "ground_truth" to calculate the accuracy.

In [30]:
import numpy as np
def cal_acc(model, input_index, output_index):
    #  _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    predicted = []
    ground_truth = [tag for seq in output_index for tag in seq]
    for i, idxs in enumerate(input_index):
        # tags_index = train_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        # targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        score, tag_seq = model(sentence_in)
        predicted.extend(tag_seq)
    accuracy = np.mean(np.array(predicted) == np.array(ground_truth))
    return predicted, ground_truth, accuracy

#### Initialize Model

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

#### Train the model

In [32]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(20):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 3247.57, train acc: 0.8506, val loss: 447.00, val acc: 0.8406, time: 26.99s
Epoch:2, Training loss: 1958.17, train acc: 0.8933, val loss: 403.80, val acc: 0.8394, time: 26.98s
Epoch:3, Training loss: 1513.78, train acc: 0.9052, val loss: 430.64, val acc: 0.8294, time: 26.96s
Epoch:4, Training loss: 1268.06, train acc: 0.9215, val loss: 432.40, val acc: 0.8319, time: 26.59s
Epoch:5, Training loss: 1109.75, train acc: 0.9218, val loss: 471.87, val acc: 0.8257, time: 26.46s
Epoch:6, Training loss: 980.30, train acc: 0.9276, val loss: 469.83, val acc: 0.8257, time: 26.83s
Epoch:7, Training loss: 855.40, train acc: 0.9373, val loss: 484.83, val acc: 0.8331, time: 26.52s
Epoch:8, Training loss: 752.97, train acc: 0.9504, val loss: 509.03, val acc: 0.8331, time: 27.79s
Epoch:9, Training loss: 668.38, train acc: 0.9577, val loss: 505.20, val acc: 0.8319, time: 29.42s
Epoch:10, Training loss: 580.82, train acc: 0.9636, val loss: 503.12, val acc: 0.8369, time: 27.72s
Epoc

### Testing

In [33]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,test_input_index,test_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))
# The log below is the sample output for this section

              precision    recall  f1-score   support

       I-LOC     0.9130    0.9333    0.9231        45
      I-MISC     0.8182    0.7200    0.7660        25
       I-ORG     0.0000    0.0000    0.0000         3
       I-PER     0.9109    0.7302    0.8106       126
           O     0.9517    0.9789    0.9651       806

    accuracy                         0.9363      1005
   macro avg     0.7188    0.6725    0.6929      1005
weighted avg     0.9387    0.9363    0.9360      1005



In [35]:
!nvidia-smi

Thu May 13 05:44:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |   1062MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------